In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
# Synthetic Data points
total_points_dataset = 1000
new_data_points = 20

# Model Parameters
num_epochs = 1500
input_dim = 1
output_dim = 1
hidden_dims = ([900, 900, 900, 900, 1200, 1600, 2400])


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Model running via: {device}")

# Synthetic data function
def F(y):
    return 0.5 * (6 * y - 2)**2 * np.sin(12 * y - 4) + 10 * (y - 0.5) - 5


# Model's total dataset
X = torch.linspace(0, 2, total_points_dataset, dtype=torch.float32)[:,None]


# Splitting dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(X, F(X), test_size=0.3, random_state=0)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=0)


# Transfering data to GPU for CUDA
X_train = X_train.to(device); X_val = X_val.to(device); X_test = X_test.to(device)
Y_train = Y_train.to(device); Y_val = Y_val.to(device); Y_test = Y_test.to(device)

In [ ]:
# Dont forget about torch data needing requires_grad = True


""" Verification Step - Can be commented out. 
import matplotlib.pyplot as plt
plt.scatter(X_val, Y_val) # main result line we want to match with
plt.show()
print(type(X_val))
"""


"""
I can use this piece of code to work on the rest of the code after taking care of training / testing 

data = np.linspace(0, 1, model_training_points)[:,None]
split = int(model_training_points * 0.8) # 80% for training, 20% for testing

train_data = data[:split]
test_data = data[split:]

plt.plot(test_data, F(test_data))
plt.show()
"""

In [ ]:
class Network(torch.nn.Module):
  def __init__(self, input_dim, HF_hidden_dims, output_dim):
    super().__init__()
    self.fc1 = torch.nn.Linear(input_dim, HF_hidden_dims[0])
    self.fc2 = torch.nn.Linear(HF_hidden_dims[0], HF_hidden_dims[1])
    self.fc3 = torch.nn.Linear(HF_hidden_dims[1], HF_hidden_dims[2])
    self.fc4 = torch.nn.Linear(HF_hidden_dims[2], HF_hidden_dims[3])
    self.fc5 = torch.nn.Linear(HF_hidden_dims[3], output_dim)
    # self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.relu(self.fc4(x))
    # x = self.dropout(x)
    x = self.fc5(x)
    return x

model = Network(input_dim, hidden_dims, output_dim).to(device)

In [ ]:
# Training
losses = [] 

# Loss criterion and optimizer
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters())


for epoch in range(num_epochs):
  y_pred = model(X_train.to(device))
  loss = criterion(y_pred, Y_train.to(device))
  losses.append(loss.item())
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

for param in model.parameters():
  param.requires_grad = True

In [ ]:
# Evaluating the network on the validation set 
val_outputs = model(X_val)
val_loss = criterion(val_outputs, Y_val)
val = print(f" Valdation Loss of Model: {val_loss}")


# Valdation Loss Plot
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
# Define the NEW input data
x_new = torch.linspace(0, 2, new_data_points)[:, None]

y_pred_new = model(x_new.to(device))


plt.plot(X, F(X)) 
plt.plot(x_new.cpu().detach().numpy(), y_pred_new.cpu().detach().numpy(),'k', lw=2, label= "NN Prediction ")
plt.plot(x_new, F(x_new), 'ro', markersize = 4, label = 'Data Points')
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='x-small')
plt.show()